In [134]:
!pip install numpy
!pip install matplotlib
!pip install PIL


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [135]:
from matplotlib.image import imread
import numpy as np
from PIL import Image

In [136]:
image = imread('data/img.png')[:, :, 0:3]
image.shape

(500, 500, 3)

In [137]:
# convert image to 8 bit per channel
image = (image * 255).astype(np.uint8)

image

array([[[190, 155,   1],
        [190, 155,   1],
        [190, 155,   1],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[190, 155,   1],
        [190, 155,   1],
        [190, 155,   1],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[189, 154,   0],
        [190, 155,   1],
        [190, 155,   1],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[237,  88,   4],
        [237,  88,   4],
        [235,  86,   4],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[237,  89,   3],
        [237,  88,   4],
        [235,  86,   4],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[237,  89,   3],
        [237,  89,   3],
        [235,  86,   4],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [138]:
with open('output/encoded_image.BMP', 'wb') as f:
    # encode BMP image
    Image.fromarray(image).save(f, 'BMP')

In [139]:
code_encode = "5739 voilà le code secret"
# convert code to binary string forcing it to be 8 bits long
caracters_values = [ord(c) for c in code_encode]
code_string = ''.join([format(c, '08b') for c in caracters_values])
print(code_string)
code_string += format(4, '08b')
code_string

00110101001101110011001100111001001000000111011001101111011010010110110011100000001000000110110001100101001000000110001101101111011001000110010100100000011100110110010101100011011100100110010101110100


'0011010100110111001100110011100100100000011101100110111101101001011011001110000000100000011011000110010100100000011000110110111101100100011001010010000001110011011001010110001101110010011001010111010000000100'

In [140]:
encoding_length = image.shape[0] * image.shape[1] * image.shape[2]
encoding_length

750000

In [141]:
def encode_bit(bit, pixel_value):
    if bit == 1:
        if pixel_value % 2 == 0:
            return pixel_value + 1
        elif pixel_value == 255:
            return pixel_value - 1
        else:
            return pixel_value
    else:
        if pixel_value % 2 == 0:
            return pixel_value
        elif pixel_value == 0:
            return pixel_value + 1
        else:
            return pixel_value - 1


In [142]:
for i, bit in enumerate(code_string):
    pixel = image[i, i % 500, 0]
    print(pixel, bit, encode_bit(int(bit), pixel))

190 0 190
190 0 190
190 1 191
191 1 191
191 0 190
192 1 193
192 0 192
192 1 193
194 0 194
194 0 194
194 1 195
194 1 195
194 0 194
194 1 195
195 1 195
195 1 195
194 0 194
194 0 194
196 1 197
196 1 197
196 0 196
196 0 196
197 1 197
197 1 197
198 0 198
154 0 154
124 1 125
134 1 135
138 1 139
135 0 134
131 0 130
128 1 129
131 0 130
123 0 122
123 1 123
120 0 120
114 0 114
113 0 112
113 0 112
112 0 112
110 0 110
110 1 111
113 1 113
115 1 115
118 0 118
123 1 123
132 1 133
129 0 128
133 0 132
144 1 145
152 1 153
154 0 154
153 1 153
152 1 153
149 1 149
146 1 147
143 0 142
143 1 143
140 1 141
140 0 140
143 1 143
142 0 142
145 0 144
152 1 153
160 0 160
159 1 159
159 1 159
157 0 156
157 1 157
158 1 159
157 0 156
153 0 152
158 1 159
159 1 159
159 1 159
160 0 160
161 0 160
161 0 160
159 0 158
157 0 156
155 0 154
153 0 152
149 1 149
143 0 142
137 0 136
132 0 132
120 0 120
102 0 102
84 0 84
72 1 73
65 1 65
61 0 60
58 1 59
58 1 59
57 0 56
50 0 50
47 0 46
39 1 39
32 1 33
42 0 42
90 0 90
144 1 145
167 0 

In [143]:
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        if i * image.shape[1] + j < len(code_string):
            for k in range(image.shape[2]):
                image[i, j, k] = encode_bit(int(code_string[i * image.shape[1] + j]), image[i, j, k])
        else:
            for k in range(image.shape[2]):
                image[i, j, k] = encode_bit(0, image[i, j, k])

with open('output/encoded_image.bmp', 'wb') as f:
    # encode BMP image
    Image.fromarray(image).save(f, 'BMP')

In [144]:
def decode_bit(pixel_value):
    return pixel_value % 2


def decode_image(image):
    k = 0
    code_string = ''
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            code_string += str(decode_bit(image[i, j, k]))
    return code_string

In [145]:
decoded_code = decode_image(image)
decoded_code

'...'

In [146]:
code_string

'0011010100110111001100110011100100100000011101100110111101101001011011001110000000100000011011000110010100100000011000110110111101100100011001010010000001110011011001010110001101110010011001010111010000000100'

In [147]:
loaded_image = imread('output/encoded_image.bmp')
loaded_image.shape
loaded_image

array([[[190, 154,   0],
        [190, 154,   0],
        [191, 155,   1],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[190, 154,   0],
        [190, 154,   0],
        [190, 154,   0],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[188, 154,   0],
        [190, 154,   0],
        [190, 154,   0],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       ...,

       [[236,  88,   4],
        [236,  88,   4],
        [234,  86,   4],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[236,  88,   2],
        [236,  88,   4],
        [234,  86,   4],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[236,  88,   2],
        [236,  88,   2],
        [234,  86,   4],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]]

In [148]:
loaded_code = decode_image(loaded_image)
loaded_code

'...'